In [ ]:
import sys
import yaml
import joblib
import os
from time import time
from tqdm import tqdm
import json
from typing import List
from dataclasses import dataclass

# TO CHANGE
BASEDIR = ".."
sys.path.insert(0, BASEDIR)

from src.pipelines.qa.answer_generator import QALLMGeneratorConfig, QALLMGenerator
from src.pipelines.qa.knowledge_retriever.utils import AbstractTripletsRetriever, BaseGraphSearchConfig
from src.db_drivers.vector_driver import VectorDBInstance
from src.kg_model import KnowledgeGraphModel
from src.utils import Logger
from src.utils.data_structs import QueryInfo, Triplet

In [ ]:
# Сделать NaiveRetriver-класс
# сделать базовый механизм кастомизации

In [ ]:
@dataclass
class NaiveGraphSearchConfig(BaseGraphSearchConfig):
    max_k: int = 50

In [ ]:
naive_search_config = NaiveGraphSearchConfig()

In [ ]:
class NaiveTripletsRetriever(AbstractTripletsRetriever):

    def __init__(self, kg_model: KnowledgeGraphModel, log: Logger, search_config: NaiveGraphSearchConfig = NaiveGraphSearchConfig(),
                 verbose: bool = False) -> None:
        self.log = log
        self.verbose = verbose
        self.kg_model = kg_model
        self.config = search_config

    def get_relevant_triplets(self, query_info: QueryInfo) -> List[Triplet]:
        query_embd = self.kg_model.embeddings_struct.embedder.encode_queries([query_info.query])[0]
        query_instance = VectorDBInstance(embedding=query_embd)

        raw_relevant_triplets = self.kg_model.embeddings_struct.vectordbs['triplets'].retrieve(
                [query_instance], self.config.max_k, includes=['metadatas'])[0]
        
        triplet_ids = list(map(lambda item: item[1].metadata['t_id'], raw_relevant_triplets))
        self.log(f"Количество извлечённых объектов из векторной бд (triplets): {len(triplet_ids)}", verbose=self.verbose)

        triplets = self.kg_model.graph_struct.db_conn.read(triplet_ids)
        self.log(f"Количество полученных трипелтов из графовой бд: {len(triplets)}", verbose=self.verbose)

        return triplets

In [ ]:
# загружаем датасет
# для каждого вопроса извлекаем N релевантных
# по id получаем триплеты
# генерируем на основании триплтов ответ